In [9]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import tensorflow_datasets as tfds
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, Dense, Flatten, Dropout, LeakyReLU, UpSampling2D, Reshape
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.preprocessing.image import array_to_img
from tensorflow.keras.callbacks import Callback

In [10]:


# Load dataset
ds, info = tfds.load('stanford_dogs', split='train', with_info=True)


In [11]:

# Define a function to scale images
def scale_image(item):
    image = item['image']
    image = tf.image.resize(image, (64, 64))
    return tf.cast(image, tf.float32) / 255.0, item['label']


In [12]:

# Preprocess dataset
ds = ds.map(scale_image)
ds = ds.cache()
ds = ds.shuffle(60000)
ds = ds.batch(batch_size=128)
ds = ds.prefetch(tf.data.experimental.AUTOTUNE)


In [13]:

# Generator model
def build_gen():
    model = Sequential()
    model.add(Dense(16*16*128, input_dim=128))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Reshape((16, 16, 128)))
    model.add(UpSampling2D())
    model.add(Conv2D(128, kernel_size=5, padding="same"))
    model.add(LeakyReLU(alpha=0.2))
    model.add(UpSampling2D())
    model.add(Conv2D(128, kernel_size=5, padding="same"))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2D(128, kernel_size=4, padding="same"))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2D(128, kernel_size=4, padding="same"))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2D(3, kernel_size=4, padding="same", activation="sigmoid"))
    return model


In [14]:

# Discriminator model
def build_disc():
    model = Sequential()
    model.add(Conv2D(64, kernel_size=5, input_shape=(64,64,3), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.4))
    model.add(Conv2D(128, kernel_size=5, padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.4))
    model.add(Conv2D(256, kernel_size=5, padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.4))
    model.add(Conv2D(512, kernel_size=5, padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.4))
    model.add(Flatten())
    model.add(Dense(1, activation="sigmoid"))
    return model


In [15]:

# GAN class
class GAN(Model):
    def __init__(self, Generator, discriminator, *args , **kwargs):
        super().__init__(*args, **kwargs)
        self.generator = Generator
        self.discriminator = discriminator

    def compile(self, g_opt, d_opt , g_loss, d_loss, *args, **kwargs):
        super().compile(*args, **kwargs)
        self.g_opt = g_opt
        self.d_opt = d_opt
        self.g_loss = g_loss
        self.d_loss = d_loss

    def train_step(self, batch_data):
        real_images, _ = batch_data
        batch_size = tf.shape(real_images)[0]
        noise = tf.random.normal((batch_size, 128))

        with tf.GradientTape() as d_tape:
            fake_images = self.generator(noise, training=False)
            yhat_real = self.discriminator(real_images, training=True)
            yhat_fake = self.discriminator(fake_images, training=True)
            yhat_realfake = tf.concat([yhat_real, yhat_fake], axis=0)
            y_realfake_labels = tf.concat([tf.zeros_like(yhat_real), tf.ones_like(yhat_fake)], axis=0)
            d_loss_value = self.d_loss(y_realfake_labels, yhat_realfake)

        d_gradient = d_tape.gradient(d_loss_value, self.discriminator.trainable_variables)
        self.d_opt.apply_gradients(zip(d_gradient, self.discriminator.trainable_variables))

        with tf.GradientTape() as g_tape:
            gen_images = self.generator(noise, training=True)
            predicted_labels = self.discriminator(gen_images, training=False)
            g_loss_value = self.g_loss(tf.ones_like(predicted_labels), predicted_labels)

        g_gradient = g_tape.gradient(g_loss_value, self.generator.trainable_variables)
        self.g_opt.apply_gradients(zip(g_gradient, self.generator.trainable_variables))

        return {"d_loss": d_loss_value, "g_loss": g_loss_value}


In [16]:
# Initialize models and optimizer
Generator = build_gen()
discriminator = build_disc()
g_opt = Adam(learning_rate=0.0002, beta_1=0.5)
d_opt = Adam(learning_rate=0.00002, beta_1=0.5)
g_loss = BinaryCrossentropy(from_logits=True)
d_loss = BinaryCrossentropy(from_logits=True)

# Compile GAN model
gan = GAN(Generator, discriminator)
gan.compile(g_opt, d_opt, g_loss, d_loss)

# Callback for generating images
class ModelMoniter(Callback):
    def __init__(self, num_img=3, latent_dim=128):
        self.num_img = num_img
        self.latent_dim = latent_dim

    def on_epoch_end(self, epoch, logs=None):
        random_latent_vectors = tf.random.normal(shape=(self.num_img, self.latent_dim))
        generated_images = self.model.generator(random_latent_vectors)
        generated_images *= 255
        generated_images = generated_images.numpy()
        for i in range(self.num_img):
            img = array_to_img(generated_images[i])
            img.save(f"generated_img{i}_{epoch}.png")


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.10/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:


# Train GAN
hist = gan.fit(ds, epochs=100, callbacks=[ModelMoniter()])

# Example usage to predict with the discriminator
discriminator.predict(img_reshaped)


Epoch 1/100


/usr/local/lib/python3.10/dist-packages/keras/src/backend/tensorflow/nn.py:681: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Sigmoid activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(
